In [210]:
import cv2
import mediapipe as mp
from matplotlib import pyplot as plt
import numpy as np
from scipy import sparse
import math
import torch
import import_ipynb
import import_ipynb
from localAngle import calc_localAngle
from coordinate import coordinateFeature
from coordinate import normFunction
from distanceNode import distance_node_feature
from local_curvature import compute_curvature
from edge_orientation_feature import get_edge_orientation

In [211]:
def JoinFeature1(NodeFeatureMatrix, EdgeFeatures, connection_tesselation):
    JoinFeatureMartix = []
    i=0
    for edge in connection_tesselation:
        JoinFeatureMartix.append(np.concatenate([NodeFeatureMatrix[edge[0]] , NodeFeatureMatrix[edge[1]] ,EdgeFeatures[i]]))
        i=i+1
    JoinFeatureMartix = np.array(JoinFeatureMartix)
    # print(JoinFeatureMartix.shape)
    return(JoinFeatureMartix)

In [212]:
def JoinFeature2(NodeFeatureMatrix, EdgeFeatures, BINARY_ADJACENCY_MATRIX):
    row_indices, col_indices = BINARY_ADJACENCY_MATRIX.nonzero()
    Joint_Features=[]
    for landmark in range(468):
        List = NodeFeatureMatrix[landmark]
        average = []
        count=0
        for i in range(len(row_indices)):
            if(row_indices[i] == landmark):
                average.append(EdgeFeatures[col_indices[i]])
                count=count+1
        transposed = [[row[i] for row in average] for i in range(len(average[0]))]
        averages = []
        for row in transposed:
            row_average = sum(row) / len(row)
            averages.append(row_average)
        Joint_Features.append(np.concatenate([List, averages]))
    Joint_Featurs=np.array(Joint_Features)
    return(Joint_Featurs)

In [213]:
def normalization_function(matrix):
    # Calculate the Euclidean norm of each row
    norms = np.linalg.norm(matrix, axis=1, keepdims=True)
    # Normalize the matrix by dividing by the norms
    return(matrix / norms)

In [214]:
def Angle(BA,BC):
    modBA= math.sqrt(BA[0]**2+BA[1]**2+BA[2]**2)
    modBC= math.sqrt(BC[0]**2+BC[1]**2+BC[2]**2)
    BABC= modBA * modBC
    dotProduct=BA[0]*BC[0] + BA[1]*BC[1] + BA[2]*BC[2]
    x = dotProduct / BABC
    angle = math.acos(x)
    return(math.degrees(angle))

In [215]:
# Calculating Local Angle For Node Feature
def fetch_local_angles(BINARY_ADJACENCY_MATRIX, x_axis, y_axis, z_axis):
    row_indices, col_indices = BINARY_ADJACENCY_MATRIX.nonzero()
    Local_Angle_features=[]
    for landmark in range(468):
        List = []
        for i in range(len(row_indices)):
            if(row_indices[i] == landmark):
                List.append([x_axis[col_indices[i]], y_axis[col_indices[i]], z_axis[col_indices[i]]])
        A = [x_axis[landmark], y_axis[landmark], z_axis[landmark]]
        # print("A=",A, "S=",List)
        Local_Angle_features.append(calc_localAngle(A, np.array(List)))
    # print("Local_Angle_features=",Local_Angle_features)
    return normFunction(Local_Angle_features)

In [216]:
# Calculating distance For Node Feature
def fetch_distance_node_feature(BINARY_ADJACENCY_MATRIX, x_axis, y_axis, z_axis):
    row_indices, col_indices = BINARY_ADJACENCY_MATRIX.nonzero()
    # print( row_indices, col_indices)
    Distance_node_features=[]
    for landmark in range(468):
        List = []
        for i in range(len(row_indices)):
            if(row_indices[i] == landmark):
                List.append([x_axis[col_indices[i]], y_axis[col_indices[i]], z_axis[col_indices[i]]])
        A = [x_axis[landmark], y_axis[landmark], z_axis[landmark]]
        Distance_node_features.append(distance_node_feature(A, np.array(List)))
    # print("Local_Angle_features=",Local_Angle_features)
    return normFunction(Distance_node_features)

In [217]:
def mediapipe(path): 
    image=cv2.imread(path)
    image=cv2.resize(image,(600,600))
    annoted_image=image.astype(np.uint8)

    mp_face_mesh=mp.solutions.face_mesh
    connection_tesselation=mp_face_mesh.FACEMESH_TESSELATION
    # print("edges_mediapipe=",len(connection_tesselation),len(connection_tesselation)/2)
    # print(connection_tesselation)
    with mp_face_mesh.FaceMesh(static_image_mode=False,max_num_faces=2,refine_landmarks=True,min_detection_confidence=0.5) as face_mesh:
        results=face_mesh.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))

    x_axis=np.empty(len(results.multi_face_landmarks[0].landmark))
    y_axis=np.empty(len(results.multi_face_landmarks[0].landmark))
    z_axis=np.empty(len(results.multi_face_landmarks[0].landmark))
    i=0

    for nodes in results.multi_face_landmarks[0].landmark:
        x_axis[i]=(results.multi_face_landmarks[0].landmark[i].x)
        y_axis[i]=(results.multi_face_landmarks[0].landmark[i].y)
        z_axis[i]=(results.multi_face_landmarks[0].landmark[i].z)
        i=i+1
    # print(len(results.multi_face_landmarks[0].landmark))
    # print("xyz=",x_axis)

    FEATURE_MATRIX=np.zeros((468,468))
    for i in range(468):
        FEATURE_MATRIX[i][i]=1

    WEIGHTED_ADJACENCY_MATRIX=np.zeros((468,468)) #creating a numpy array of shape 468X468 initialized with zero
    distance_feature=[]
    BINARY_ADJACENCY_MATRIX=np.zeros((468,468))
    ANGLE_BETWEEN_EDGE_FEATURE=[]
    LOCAL_CURVATURE_FEATURE=[]
    EDGE_ORIENTATION_FEATURE=[]

    for edge in connection_tesselation:
        # Determining X & Y axis of Two Connected Points
        x1=x_axis[edge[0]]
        y1=y_axis[edge[0]]
        z1=z_axis[edge[0]]
        x2=x_axis[edge[1]]
        y2=y_axis[edge[1]]
        z2=z_axis[edge[1]]
        # Calculating Eucleadin Distance for Weighted Graph
        eucleadian_distance=(((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2) ** 0.5)
        # Creating Weighted Graph
        WEIGHTED_ADJACENCY_MATRIX[edge[0]][edge[1]]=eucleadian_distance
        WEIGHTED_ADJACENCY_MATRIX[edge[1]][edge[0]]=eucleadian_distance
        distance_feature.append(eucleadian_distance)

        BINARY_ADJACENCY_MATRIX[edge[0]][edge[1]]=1
        BINARY_ADJACENCY_MATRIX[edge[1]][edge[0]]=1

        angle_betn_edge=Angle([x1, y1, z1],[x2, y2, z2])
        ANGLE_BETWEEN_EDGE_FEATURE.append(angle_betn_edge)
        # ANGLE_BETWEEN_EDGE_FEATURE[edge[1]][edge[0]]=angle_betn_edge

        local_curvature=compute_curvature(x_axis, y_axis, z_axis, edge[0], edge[1])
        LOCAL_CURVATURE_FEATURE.append(local_curvature)
        # LOCAL_CURVATURE_FEATURE[edge[1]][edge[0]]=local_curvature
        # print(local_curvature)
        edge_orientation_feature=get_edge_orientation(((x1,y1,z1), (x2,y2,z2)))
        EDGE_ORIENTATION_FEATURE.append(edge_orientation_feature)
        # EDGE_ORIENTATION_FEATURE[edge[1]][edge[0]]=edge_orientation_feature
        # Normalizing axis Values For Visualizing Meash on Face
        shape = image.shape 
        relative_x = int(x1 * shape[1])
        relative_y = int(y1 * shape[0])
        relative_z = int(x2 * shape[1])
        relative_c = int(y2 * shape[0])
        # Drawing Nodes and Edges on Image For Visualization
        cv2.circle(image, (relative_x, relative_y), radius=1, color=(0, 0, 255), thickness=3) 
        cv2.line(image,(relative_x,relative_y),(relative_z,relative_c),(0,255,0),1)

    EdgeFeatures = normalization_function(np.column_stack(( distance_feature,  ANGLE_BETWEEN_EDGE_FEATURE, LOCAL_CURVATURE_FEATURE,EDGE_ORIENTATION_FEATURE)))

    LocalAngles = fetch_local_angles(BINARY_ADJACENCY_MATRIX, x_axis, y_axis, z_axis)
    Distance_node_feature = fetch_distance_node_feature(BINARY_ADJACENCY_MATRIX, x_axis, y_axis, z_axis)
    XCoordinateFeature, YCoordinateFeature, ZCoordinateFeature = coordinateFeature(x_axis, y_axis, z_axis)
    NodeFeatureMatrix = np.column_stack(( LocalAngles,  XCoordinateFeature[:-10], YCoordinateFeature[:-10], ZCoordinateFeature[:-10], Distance_node_feature, FEATURE_MATRIX))
    
    # JoinFeatureMatrix = normalization_function(JoinFeature1(NodeFeatureMatrix, EdgeFeatures, connection_tesselation))
    JoinFeatureMatrix = JoinFeature2(NodeFeatureMatrix, EdgeFeatures, BINARY_ADJACENCY_MATRIX)

    WEIGHTED_ADJACENCY_MATRIX=sparse.csr_matrix(WEIGHTED_ADJACENCY_MATRIX)
    BINARY_ADJACENCY_MATRIX=sparse.csr_matrix(BINARY_ADJACENCY_MATRIX)
    # print("NodeFeatureMatrix",NodeFeatureMatrix.shape, "EdgeFeatures", EdgeFeatures.shape)
    # print("JoinFeatureMatrix shape=", JoinFeatureMatrix.shape)
    # for i in range(10):
    #     # print("NodeFeatureMatrix",NodeFeatureMatrix[i], "EdgeFeatures", EdgeFeatures[i])
    #     print("JoinFeatureMatrix", JoinFeatureMatrix[i])
    return(WEIGHTED_ADJACENCY_MATRIX, torch.from_numpy(JoinFeatureMatrix))


Checking mediapipe is running currectly or not

In [218]:
# path="./datasets/orl_dataset/person10/test_image/98_10.jpg"
# adj1,features1=mediapipe(path)

JoinFeatureMatrix shape= (468, 9)
JoinFeatureMatrix [-0.77592709 -0.02354432  0.50575377 -0.49848654 -0.65849809  0.00212274
  0.10331858  0.24562892  0.89508562]
JoinFeatureMatrix [-0.54242812  0.00692396  0.34259127 -0.93040907 -0.72743056  0.00096948
  0.05205974  0.4745478   0.78421699]
JoinFeatureMatrix [ 6.07223401e-01 -3.03253315e-03  3.77432038e-01 -5.85335154e-01
 -5.17783657e-01  6.42778955e-04  3.33304814e-02  1.52749234e-01
  9.62477933e-01]
JoinFeatureMatrix [-4.40377351e-01 -5.40066769e-02  4.69430164e-02 -8.90904648e-01
 -5.17556752e-01  5.11626468e-04  2.70404745e-02  1.72469579e-01
  9.66510031e-01]
JoinFeatureMatrix [-6.98995698e-01  1.28099556e-02  2.67581130e-01 -1.00000000e+00
 -4.79607905e-01  3.68109774e-04  2.30275076e-02  1.84661939e-01
  9.22491552e-01]
JoinFeatureMatrix [-7.41909454e-01  1.93308733e-02  1.47972264e-01 -9.96533111e-01
 -5.59601178e-01  9.67250960e-04  5.51740716e-02  1.21820495e-01
  9.87068247e-01]
JoinFeatureMatrix [-1.44474602e-01  3.191062

In [219]:
# path="./datasets/orl_dataset/person9/test_image/88_9.jpg"
# adj1,features1=mediapipe(path)

In [220]:
# path="./datasets/orl_dataset/person9/test_image/88_9.jpg"
# adj1,features1=mediapipe(path)
# for i in range(50):
#     print(features1[i])